Title: Popular Classification Algorithms

Support Vector Machines (SVM)

Task 1: Identify handwriting on checks and classify each letter.

In [5]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

# -----------------------------------------
# STEP 1: Train or Load the EMNIST Model
# -----------------------------------------

def train_emnist_model():
    import tensorflow_datasets as tfds
    (ds_train, ds_test), ds_info = tfds.load('emnist/letters', split=['train', 'test'],
                                             with_info=True, as_supervised=True)

    def normalize_img(image, label):
        return tf.cast(image, tf.float32) / 255.0, label

    ds_train = ds_train.map(normalize_img).batch(128).prefetch(1)
    ds_test = ds_test.map(normalize_img).batch(128).prefetch(1)

    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(27, activation='softmax')  # 26 letters + label 0
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print("Training EMNIST model (this may take a few minutes)...")
    model.fit(ds_train, validation_data=ds_test, epochs=5)
    model.save('emnist_cnn.h5')
    print("Model saved as emnist_cnn.h5")
    return model

# Load or train model
if os.path.exists('emnist_cnn.h5'):
    model = tf.keras.models.load_model('emnist_cnn.h5')
else:
    model = train_emnist_model()

# Label map (EMNIST letters start from 1 to 26, mapping to 'A' to 'Z')
emnist_labels = [''] + [chr(i) for i in range(65, 91)]

# -----------------------------------------
# STEP 2: Preprocess the Check Image
# -----------------------------------------

def preprocess_check_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image not found.")
    img = cv2.resize(img, (800, 400))  # Resize for consistency
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return img, thresh

# -----------------------------------------
# STEP 3: Segment Characters
# -----------------------------------------

def extract_characters(thresh_img):
    contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    characters = []
    bounding_boxes = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if 10 < w < 100 and 15 < h < 100:
            char = thresh_img[y:y+h, x:x+w]
            resized = cv2.resize(char, (28, 28), interpolation=cv2.INTER_AREA)
            characters.append(resized)
            bounding_boxes.append(x)

    # Sort characters left to right
    characters = [char for _, char in sorted(zip(bounding_boxes, characters))]
    return characters

# -----------------------------------------
# STEP 4: Predict Characters
# -----------------------------------------

def predict_characters(characters, model):
    result = ""
    for char_img in characters:
        norm_img = char_img.astype("float32") / 255.0
        norm_img = np.expand_dims(norm_img, axis=(0, -1))  # shape (1, 28, 28, 1)
        prediction = model.predict(norm_img, verbose=0)
        label = np.argmax(prediction)
        result += emnist_labels[label]
    return result

# -----------------------------------------
# STEP 5: Run the Full Pipeline
# -----------------------------------------

def main(image_path):
    try:
        original, thresh = preprocess_check_image(image_path)
        characters = extract_characters(thresh)
        if not characters:
            print("No characters found. Try another image.")
            return
        result = predict_characters(characters, model)
        print("Predicted Text:", result)

        # Optional: display results
        cv2.imshow("Check Image", original)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    except Exception as e:
        print("Error:", e)

# -----------------------------------------
# Entry Point
# -----------------------------------------

if __name__ == "__main__":
    # Replace with the path to your check image
    image_path = "check_sample.png"
    main(image_path)


2025-05-19 05:52:27.134152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:23<00:00, 10.54s/ url]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]2025-05-19 05:52:59.589651: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Dataset emnist downloaded and prepared to /home/vscode/tensorflow_datasets/emnist/letters/3.1.0. Subsequent calls will reuse this data.
Training EMNIST model (this may take a few minutes)...
Epoch 1/5


2025-05-19 05:53:26.152236: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


694/694 ━━━━━━━━━━━━━━━━━━━━ 51s 71ms/step - accuracy: 0.6768 - loss: 1.1076 - val_accuracy: 0.8780 - val_loss: 0.3741
Epoch 2/5
694/694 ━━━━━━━━━━━━━━━━━━━━ 43s 61ms/step - accuracy: 0.9008 - loss: 0.3071 - val_accuracy: 0.8997 - val_loss: 0.3021
Epoch 3/5
694/694 ━━━━━━━━━━━━━━━━━━━━ 43s 62ms/step - accuracy: 0.9204 - loss: 0.2384 - val_accuracy: 0.9109 - val_loss: 0.2700
Epoch 4/5
694/694 ━━━━━━━━━━━━━━━━━━━━ 81s 60ms/step - accuracy: 0.9318 - loss: 0.2005 - val_accuracy: 0.9150 - val_loss: 0.2582
Epoch 5/5
694/694 ━━━━━━━━━━━━━━━━━━━━ 41s 60ms/step - accuracy: 0.9401 - loss: 0.1740 - val_accuracy: 0.9166 - val_loss: 0.2587


Model saved as emnist_cnn.h5
Error: Image not found.


[ WARN:0@344.674] global loadsave.cpp:268 findDecoder imread_('check_sample.png'): can't open/read file: check file path/integrity



Task 2: Detect gender of a speaker based on voice data.

In [9]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
import soundfile as sf

# ------------------------------
# STEP 1: Feature Extraction
# ------------------------------
male_01.wav
female_01.wav
male_02.wav
female_02.wav

def extract_features(file_path, sr=22050):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    y = librosa.util.normalize(y)

    # Extract features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

    # Average over time axis
    features = np.hstack([
        np.mean(mfccs, axis=1),
        np.mean(chroma, axis=1),
        np.mean(spec_contrast, axis=1)
    ])
    return features

# ------------------------------
# STEP 2: Prepare Dataset
# ------------------------------

def load_dataset(folder_path):
    X, y = [], []
    for file in os.listdir(folder_path):
        if file.endswith(".wav"):
            label = 'male' if 'male' in file.lower() else 'female'
            file_path = os.path.join(folder_path, file)
            features = extract_features(file_path)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

# ------------------------------
# STEP 3: Train the Model
# ------------------------------

def train_model(X, y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

    model = SVC(kernel='rbf', probability=True)
    model.fit(X_train, y_train)

    print(f"Model accuracy: {model.score(X_test, y_test) * 100:.2f}%")
    return model, scaler, le

# ------------------------------
# STEP 4: Predict Gender
# ------------------------------

def predict_gender(file_path, model, scaler, label_encoder):
    features = extract_features(file_path)
    features_scaled = scaler.transform([features])
    prediction = model.predict(features_scaled)
    return label_encoder.inverse_transform(prediction)[0]

# ------------------------------
# MAIN EXECUTION
# ------------------------------

if __name__ == "__main__":
    # 📂 Your dataset folder should contain labeled .wav files like:
    # 'female_01.wav', 'male_02.wav', etc.
    folder = "voice_dataset"

    X, y = load_dataset(folder)
    model, scaler, label_encoder = train_model(X, y)

    # 🔍 Predict on a new sample
    test_file = "test_voice.wav"
    if os.path.exists(test_file):
        gender = predict_gender(test_file, model, scaler, label_encoder)
        print("Predicted Gender:", gender)
    else:
        print("Test file not found:", test_file)


NameError: name 'male_01' is not defined

Task 3: Classify email topics based on content.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import nltk
import re

nltk.download('stopwords')
from nltk.corpus import stopwords

# -----------------------------
# STEP 1: Sample Email Dataset
# -----------------------------

data = {
    'email': [
        "Meeting at 10am tomorrow. Bring the budget report.",
        "Your flight has been booked. Please check your itinerary.",
        "Earn money fast! Get rich with this one simple trick.",
        "Invoice for last month attached. Please process the payment.",
        "Reminder: Submit your assignment by Friday.",
        "50% OFF your favorite items! Limited-time offer!",
        "Your bank statement is now available online.",
        "Let's catch up this weekend for lunch."
    ],
    'topic': [
        'Work',
        'Travel',
        'Spam',
        'Finance',
        'Work',
        'Promotion',
        'Finance',
        'Personal'
    ]
}

df = pd.DataFrame(data)

# -----------------------------
# STEP 2: Clean Text
# -----------------------------

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation
    return text

df['cleaned'] = df['email'].apply(clean_text)

# -----------------------------
# STEP 3: Train the Classifier
# -----------------------------

X = df['cleaned']
y = df['topic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = make_pipeline(
    TfidfVectorizer(stop_words=stopwords.words('english')),
    MultinomialNB()
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# -----------------------------
# STEP 4: Predict New Email
# -----------------------------

def classify_email(content):
    cleaned = clean_text(content)
    return model.predict([cleaned])[0]

# Test
test_email = "Don't forget the team meeting on Monday at 9am."
print("\nPredicted Topic:", classify_email(test_email))


Classification Report:

              precision    recall  f1-score   support

     Finance       0.00      0.00      0.00       0.0
   Promotion       0.00      0.00      0.00       1.0
      Travel       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0


Predicted Topic: Work


[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/vscode/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vscode/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vscode/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to 